In [1]:
import pandas as pd
from Functions import *
import scipy.stats as stats

In [2]:
data_tpts, data_x, data_y = data_deal()
MISE, index_z = compared(n = 10, seed_z = 1, data_x = data_x, data_y = data_y, data_tpts = data_tpts, 
                   n_basis = 4, order = 3, test_size = 0.2, plot_flag=False,
                   method_list = ['lin_b', 'knn_b', 'gbdt_b', 'xgb_b', 'MBR_XGB'])

Data processing completed!


In [3]:
df = pd.DataFrame(index_z)
# 保留第一列两位小数
df['R2_var'] = np.sqrt(df['R2_var'])
df['Mise_var'] = np.sqrt(df['Mise_var'])
df['Rmse_var'] = np.sqrt(df['Rmse_var'])
df=df.round(3)
print(df)
# df.to_csv('results.csv', index=False)

  method_name  R2_mean  R2_var  Mise_mean  Mise_var  Rmse_mean  Rmse_var
0       lin_b    0.860   0.134      0.499     0.520      0.122     0.068
1       knn_b    0.890   0.106      0.401     0.437      0.110     0.067
2      gbdt_b    0.898   0.105      0.378     0.420      0.098     0.072
3       xgb_b    0.897   0.106      0.379     0.413      0.100     0.069
4     MBR_XGB    0.905   0.106      0.357     0.418      0.095     0.070


In [4]:
MISE = pd.DataFrame(MISE)
print(MISE)

        LIN       KNN      GBDT       XGB   MBR-XGB
0  0.093960  0.027854  0.001266  0.003281  0.004596
1  0.640866  0.314460  0.545817  0.621829  0.473267
2  0.014036  0.016796  0.013306  0.012876  0.016815
3  0.028136  0.044643  0.007338  0.010998  0.011699
4  0.072858  0.027430  0.029030  0.035101  0.021240
5  0.967830  1.057530  0.873218  0.829704  0.772944
6  1.283685  0.974347  0.963456  0.983860  0.980472
7  0.082074  0.165830  0.188256  0.170646  0.171675
8  1.424985  1.120230  1.078533  1.027804  1.098158
9  0.380851  0.264990  0.082376  0.090538  0.019540


## Friedman test and Nemenyi test

In [5]:
data = MISE
test_data = data.values.T
# 计算秩
ranks = np.empty(data.values.shape)
for i in range(data.values.shape[0]):
    ranks[i, :] = stats.rankdata(data.values[i, :], method='min')
# 将秩转换为DataFrame
ranks_df = pd.DataFrame(ranks, columns=data.columns, index=range(1, len(ranks) + 1))
ranks_df.loc['Mean'] = ranks_df.mean(axis=0)
print(ranks_df)

# 执行Friedman检验
chi2, p_value = stats.friedmanchisquare(*test_data)
print(f"Chi-squared: {chi2}, P-value: {p_value}")

      LIN  KNN  GBDT  XGB  MBR-XGB
1     5.0  4.0   1.0  2.0      3.0
2     5.0  1.0   3.0  4.0      2.0
3     3.0  4.0   2.0  1.0      5.0
4     4.0  5.0   1.0  2.0      3.0
5     5.0  2.0   3.0  4.0      1.0
6     4.0  5.0   3.0  2.0      1.0
7     5.0  2.0   1.0  4.0      3.0
8     1.0  2.0   5.0  3.0      4.0
9     5.0  4.0   2.0  1.0      3.0
10    5.0  4.0   2.0  3.0      1.0
Mean  4.2  3.3   2.3  2.6      2.6
Chi-squared: 9.360000000000014, P-value: 0.052704798878527435


In [6]:
import scikit_posthocs as sp
result = sp.posthoc_nemenyi_friedman(data)
print(result.round(4))

            LIN     KNN    GBDT     XGB  MBR-XGB
LIN      1.0000  0.7080  0.0559  0.1571   0.1571
KNN      0.7080  1.0000  0.6184  0.8600   0.8600
GBDT     0.0559  0.6184  1.0000  0.9932   0.9932
XGB      0.1571  0.8600  0.9932  1.0000   1.0000
MBR-XGB  0.1571  0.8600  0.9932  1.0000   1.0000
